In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

In [2]:
mnist = input_data.read_data_sets('data/', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
# 入力データを定義
x = tf.placeholder(tf.float32, [None, 784])

In [4]:
# 入力画像をログに出力
img = tf.reshape(x, [-1, 28, 28, 1])
tf.summary.image('input_data', img, 10)

<tf.Tensor 'input_data:0' shape=() dtype=string>

In [5]:
# 入力層から中間層
with tf.name_scope('hidden'):
    w_1 = tf.Variable(tf.truncated_normal([784, 64], stddev=0.1), name='w1')
    b_1 = tf.Variable(tf.zeros([64]), name='b1')
    h_1 = tf.nn.relu(tf.matmul(x, w_1) + b_1)

    # 中間層の重みの分布をログ出力
    tf.summary.histogram('w_1', w_1)

In [6]:
# 中間層から出力層
with tf.name_scope('output'):
    w_2 = tf.Variable(tf.truncated_normal([64, 10], stddev=0.1), name='w2')
    b_2 = tf.Variable(tf.zeros([10]), name='b2')
    out = tf.nn.softmax(tf.matmul(h_1, w_2) + b_2)

In [7]:
# 誤差関数
y = tf.placeholder(tf.float32, [None, 10])
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.square(y - out))
    tf.summary.scalar('loss', loss)

In [8]:
# 訓練
with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

In [9]:
# 評価
with tf.name_scope('accuracy'):
    correct = tf.equal(tf.argmax(out, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    tf.summary.scalar('accuracy', accuracy)

In [10]:
# ログのマージ
summary_op = tf.summary.merge_all()

In [11]:
# 変数の初期化
init = tf.global_variables_initializer()

In [12]:
# 実行
with tf.Session() as sess:
    summary_writer = tf.summary.FileWriter('logs', sess.graph)

    sess.run(init)

    # テストデータのロード
    test_images = mnist.test.images
    test_labels = mnist.test.labels

    for i in range(1000):
        step = i + 1
        train_images, train_labels = mnist.train.next_batch(50)
        sess.run(train_step, feed_dict={x: train_images, y: train_labels})

        if step % 10 == 0:
            # ログを取る処理を実行する（出力はログ情報が書かれたプロトコルバッファ）
            summary_str = sess.run(summary_op, feed_dict={x: test_images, y: test_labels})
            # ログ情報のプロトコルバッファを書き込む
            summary_writer.add_summary(summary_str, step)

# TensorBoardの確認

コマンドプロンプトから下記コマンドを実行する
```
$ tensorboard --logdir ./logs
```

実行結果は下記の通り

![SCALARS](./images/tensorboard_01_scalars.PNG "SCALARS")

![IMAGES](./images/tensorboard_02_images.PNG "IMAGES")

![GRAPHS](./images/tensorboard_03_graphs.PNG "GRAPHS")

![DISTRIBUTIONS](./images/tensorboard_04_distributions.PNG "DISTRIBUTIONS")

![HISTGRAMS](./images/tensorboard_05_histgrams.PNG "HISTGRAMS")